In [1]:
import pandas as pd
import numpy as np
import pycaret.classification as pycc
from sklearn.metrics import f1_score,log_loss
from pycaret.classification import * 
import requests

# teste API

In [21]:
dados = catalog.load('primary_data_3PT').iloc[5].to_dict()
dados

[04/15/23 19:57:33] INFO     Loading data from 'primary_data_3PT' (ParquetDataSet)...           data_catalog.py:343

{'lat': 34.0443,
 'lon': -118.2698,
 'minutes_remaining': 8.0,
 'period': 3.0,
 'playoffs': 0.0,
 'shot_distance': 0.0,
 'shot_made_flag': 1.0}

In [22]:
url = 'http://127.0.0.1:5001/invocations'

data = {
    "dataframe_records": [
{'lat': 34.0443,
 'lon': -118.2698,
 'minutes_remaining': 8.0,
 'period': 3.0,
 'playoffs': 0.0,
 'shot_distance': 0.0}
    ]
}

results = requests.post(url,json = data)
results.status_code

200

In [23]:
results.json()

{'predictions': [1.0]}

# testes modelos

In [2]:
dados = catalog.load('data_train')

[04/15/23 18:54:39] INFO     Loading data from 'data_train' (ParquetDataSet)...                 data_catalog.py:343

In [3]:
pycc.setup(dados, target = 'shot_made_flag')
add_metric('logloss', 'Log Loss', log_loss, greater_is_better=False)

,Description,Value
0,Session id,7116
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(16228, 7)"
4,Transformed data shape,"(16228, 7)"
5,Transformed train set shape,"(11359, 7)"
6,Transformed test set shape,"(4869, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


Name                                                       Log Loss
Display Name                                               Log Loss
Score Function            <function log_loss at 0x000002D4AA764700>
Scorer               make_scorer(log_loss, greater_is_better=False)
Target                                                         pred
Args                                                             {}
Greater is Better                                             False
Multiclass                                                     True
Custom                                                         True
Name: logloss, dtype: object

In [5]:
com_model = pycc.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Log Loss,TT (Sec)
ada,Ada Boost Classifier,0.5929,0.5995,0.3866,0.6182,0.4753,0.1708,0.1836,6.4898,0.6830
gbc,Gradient Boosting Classifier,0.5862,0.5971,0.3851,0.6050,0.4704,0.1576,0.1685,6.5964,0.8930
ridge,Ridge Classifier,0.5766,0.0000,0.4860,0.5661,0.5230,0.1464,0.1478,6.7494,0.5370
lda,Linear Discriminant Analysis,0.5765,0.5996,0.4864,0.5658,0.5230,0.1460,0.1474,6.7522,0.5570
lr,Logistic Regression,0.5749,0.5977,0.4818,0.5643,0.5197,0.1426,0.1441,6.7775,1.2440
nb,Naive Bayes,0.5745,0.5959,0.5079,0.5601,0.5327,0.1439,0.1445,6.7831,0.5270
qda,Quadratic Discriminant Analysis,0.5723,0.6005,0.5341,0.5542,0.5440,0.1416,0.1417,6.8182,0.4560
lightgbm,Light Gradient Boosting Machine,0.5695,0.5870,0.4731,0.5583,0.5118,0.1316,0.1331,6.8631,0.7600
et,Extra Trees Classifier,0.5537,0.5663,0.5389,0.5326,0.5356,0.1062,0.1063,7.1143,2.3330
rf,Random Forest Classifier,0.5523,0.5680,0.5144,0.5324,0.5231,0.1016,0.1017,7.1368,1.8930


In [5]:
lr_clf = pycc.create_model('lr', fold = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Log Loss
Fold,,,,,,,,
0,0.5898,0.6073,0.4885,0.5843,0.5321,0.1721,0.1743,6.5397
1,0.5673,0.5933,0.4498,0.5584,0.4982,0.1256,0.1279,6.8976
2,0.5669,0.5903,0.4627,0.5559,0.5050,0.1258,0.1274,6.9046
3,0.5629,0.5745,0.4673,0.5499,0.5052,0.1184,0.1197,6.9678
4,0.5848,0.6061,0.5129,0.5726,0.5411,0.1640,0.1649,6.6198
Mean,0.5743,0.5943,0.4762,0.5642,0.5163,0.1412,0.1428,6.7859
Std,0.0108,0.0120,0.0222,0.0125,0.0170,0.0222,0.0222,0.1719


In [47]:
# Make model predictions on unseen data
stacked_clf_dynamic_pred_unseen = pycc.predict_model(lr_clf,
                                                     raw_score = True,
                                                     data = catalog.load('data_test'))

[04/09/23 00:33:01] INFO     Loading data from 'data_test' (ParquetDataSet)...                  data_catalog.py:343

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Log Loss
0,Logistic Regression,0.5783,0.5976,0.4995,0.5652,0.5303,0.1503,0.1512,6.7235


In [48]:
stacked_clf_dynamic_pred_results = pycc.pull()
stacked_clf_dynamic_pred_results['F1']

0    0.5303
Name: F1, dtype: float64

In [49]:
stacked_clf_dynamic_pred_results['Log Loss']

0    6.7235
Name: Log Loss, dtype: float64

In [6]:
available_models = pycc.models()
available_models

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_clf = pycc.compare_models(cross_validation=False)

In [ ]:
best_clf = pycc.compare_models()

In [ ]:
print(best_clf)

In [ ]:
best_clf_specific = pycc.compare_models(include = ['ada', 'qda', 'svm'])

In [ ]:
pycc.plot_model(best_clf_specific, plot = 'auc')

In [ ]:
pycc.plot_model(best_clf_specific, 
                plot = 'confusion_matrix', 
                plot_kwargs = {'percent' : True})

In [ ]:
ada_clf = pycc.create_model('ada', fold = 5)

In [19]:
ada_clf_results = pycc.pull()
ada_clf_results.loc[['Mean', 'Std']]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
Mean,0.5915,0.6053,0.3826,0.6164,0.4720,0.1678,0.1805
Std,0.0059,0.0074,0.0160,0.0078,0.0135,0.0126,0.0124


In [20]:
ada_clf_tuned = pycc.tune_model(ada_clf, n_iter = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5766,0.5666,0.3487,0.5962,0.4400,0.1357,0.1484
1,0.5942,0.5838,0.3579,0.6319,0.4570,0.1709,0.1886
2,0.6224,0.6127,0.4022,0.6749,0.5040,0.2295,0.2496
3,0.5827,0.5731,0.3635,0.6043,0.4539,0.1489,0.1615
4,0.5977,0.5891,0.4004,0.6218,0.4871,0.1810,0.1929
5,0.5924,0.5820,0.3444,0.6361,0.4468,0.1672,0.1870
6,0.6004,0.5899,0.3517,0.6519,0.4569,0.1833,0.2052
7,0.6012,0.5914,0.3683,0.6452,0.4689,0.1862,0.2050
8,0.5995,0.5889,0.3481,0.6517,0.4538,0.1813,0.2036


Fitting 10 folds for each of 5 candidates, totalling 50 fits


In [22]:
ada_clf_tuned_better_tuner = pycc.tune_model(ada_clf,n_iter = 5, 
                                            return_tuner=True,
                                            choose_better = True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5766,0.5666,0.3487,0.5962,0.4400,0.1357,0.1484
1,0.5942,0.5838,0.3579,0.6319,0.4570,0.1709,0.1886
2,0.6224,0.6127,0.4022,0.6749,0.5040,0.2295,0.2496
3,0.5827,0.5731,0.3635,0.6043,0.4539,0.1489,0.1615
4,0.5977,0.5891,0.4004,0.6218,0.4871,0.1810,0.1929
5,0.5924,0.5820,0.3444,0.6361,0.4468,0.1672,0.1870
6,0.6004,0.5899,0.3517,0.6519,0.4569,0.1833,0.2052
7,0.6012,0.5914,0.3683,0.6452,0.4689,0.1862,0.2050
8,0.5995,0.5889,0.3481,0.6517,0.4538,0.1813,0.2036


Fitting 10 folds for each of 5 candidates, totalling 50 fits


In [27]:
ada_clf

AdaBoostClassifier(algorithm='SAMME.R', base_estimator='deprecated',
                   estimator=None, learning_rate=1.0, n_estimators=50,
                   random_state=3623)

In [ ]:
ada_clf_tuned_better_results = pycc.pull()

In [25]:
stacked_clf_dynamic_pred_unseen = pycc.predict_model(ada_clf,
                                                     raw_score = True,
                                                     data = catalog.load('data_test'))

stacked_clf_dynamic_pred_unseen

[04/08/23 02:46:29] INFO     Loading data from 'data_test' (ParquetDataSet)...                  data_catalog.py:343

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,str,0.5953,0.5976,0.3930,0.6189,0.4807,0.1754,0.1876


,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag,prediction_label,prediction_score_0,prediction_score_1
6631,34.036301,-118.302803,8,2,0,3,0.0,0,0.5011,0.4989
10543,34.024300,-118.287804,6,4,0,2,1.0,1,0.4975,0.5025
12944,34.044300,-118.269798,7,2,0,0,0.0,1,0.4965,0.5035
18074,34.007301,-118.169800,5,1,0,10,0.0,0,0.5008,0.4992
5033,33.903301,-118.129799,4,1,0,19,1.0,0,0.5019,0.4981
...,...,...,...,...,...,...,...,...,...,...
3952,34.044300,-118.269798,9,2,0,0,1.0,1,0.4968,0.5032
100,33.894299,-118.245796,9,3,0,15,0.0,0,0.5006,0.4994
30058,34.044300,-118.269798,7,4,1,0,1.0,1,0.4970,0.5030
24039,33.955299,-118.268799,11,2,0,8,0.0,0,0.5010,0.4990


In [1]:
import shap

[04/08/23 23:22:05] WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=802993;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py\warnings.py]8;;\:]8;id=515620;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

                    WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=798025;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py\warnings.py]8;;\:]8;id=561168;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

                    WARNING  Importing display from IPython.core.display is deprecated since        ]8;id=414867;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py\warnings.py]8;;\:]8;id=917600;file://C:\Users\User\AppData\Local\Programs\Python\Python310\lib\warnings.py#109\109]8;;\
                             IPython 7.14, please import from IPython display                                      
                                                                                                                   

In [3]:
model = catalog.load('final_model_lr')

[04/08/23 23:22:57] INFO     Loading data from 'final_model_lr'                                 data_catalog.py:343
                             (MlflowMlflowModelSaverDataSet)...                                                    

In [4]:
explainer = shap.Explainer(model.predict)

In [19]:
data_test = catalog.load('data_test')
data_test

[04/09/23 02:17:00] INFO     Loading data from 'data_test' (ParquetDataSet)...                  data_catalog.py:343

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
6631,34.0363,-118.3028,8,2,0,3,0.0
10543,34.0243,-118.2878,6,4,0,2,1.0
12944,34.0443,-118.2698,7,2,0,0,0.0
18074,34.0073,-118.1698,5,1,0,10,0.0
5033,33.9033,-118.1298,4,1,0,19,1.0
...,...,...,...,...,...,...,...
3952,34.0443,-118.2698,9,2,0,0,1.0
100,33.8943,-118.2458,9,3,0,15,0.0
30058,34.0443,-118.2698,7,4,1,0,1.0
24039,33.9553,-118.2688,11,2,0,8,0.0


In [24]:
data_test[['shot_made_flag']]

,shot_made_flag
6631,0.0
10543,1.0
12944,0.0
18074,0.0
5033,1.0
...,...
3952,1.0
100,0.0
30058,1.0
24039,0.0


In [16]:
model = catalog.load('final_model_lr')

[04/09/23 02:15:48] INFO     Loading data from 'final_model_lr' (PickleDataSet)...              data_catalog.py:343

In [17]:
model

LogisticRegression(max_iter=1000, random_state=8087)

In [30]:
X = data_test.drop(columns=['shot_made_flag']).copy()
y_pred = model.predict(X)

In [31]:
y_true = data_test['shot_made_flag']

In [ ]:
y_true

In [33]:
f1 = f1_score(y_true, y_pred)
f1

0.5312843029637759